In [1]:
#import libraries
import torch
import torch.nn as nn
import shutil
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset,DataLoader,random_split
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
from PIL import Image,ImageFile
import torch.optim as optim
import pandas as pd
import copy
from torch.optim import lr_scheduler

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [19]:
base_data_dir = '../data'
fake_dir = os.path.join(base_data_dir, 'generated')  # Fake artworks directory
real_dir = os.path.join(base_data_dir, 'real')  # Real artworks directory

ARTWORK DATASET

In [20]:
#Class to manage artworks with respect to their authenticity

class ArtworkDataset(Dataset):
  def __init__(self,links,transform):
      self.data = links
      self.transform = transform

  def __len__(self):
    return self.data.index.shape[0]
    
  def __getitem__(self,idx):
        img = Image.open(self.data.iloc[idx,0])
        label_index = self.data.iloc[idx, 1]
        if (img.mode != 'RGB'):
            img = img.convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label_index


In [38]:
# Create a CSV file with paths to artwork images and their labels (real or fake)
data = [] 

# Iterate over the fake artworks and add their paths and labels to the list
for dirpath, dirnames, filenames in os.walk(fake_dir):
    for filename in filenames:
        if filename.endswith(".jpg"): # only consider jpg files
            filepath = os.path.join(dirpath, filename)
            data.append((filepath, "0"))


# Iterate over the real artworks and add their paths and labels to the list
for dirpath, dirnames, filenames in os.walk(real_dir):
    for filename in filenames:
        if filename.endswith(".jpg"):
            filepath = os.path.join(dirpath, filename)
            data.append((filepath, "1"))  # Label 1 for real artworks

# Convert the list "data" to a pandas dataframe
df = pd.DataFrame(data, columns=["path", "label"])

# Save the dataframe to a CSV file
csv_output_path = os.path.join(base_data_dir, "image_labels.csv")
df.to_csv(csv_output_path, index=False)
print(f"CSV file saved at {csv_output_path}")


CSV file saved at ../data/image_labels.csv


LOAD PRETRAINED MODEL

In [39]:
%pip install timm
import timm

model = timm.create_model('resnet50',pretrained=True, num_classes=2)

model = model.to(device)
print(model)

Note: you may need to restart the kernel to use updated packages.
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps

In [40]:
#Setting the model weights to non-trainable
for param in model.parameters(): 
    param.requires_grad = False

In [41]:
#Make the last layer of the model trainable
for p in model.fc.parameters(): 
    p.requires_grad=True


SPLIT IN TRAINING AND VALIDATION SET

In [42]:
dataset = df
dataset['label'] = dataset['label'].astype(int)
dataset

,path,label
0,../data/generated/stylegan3-t-metfaces-1024x10...,0
1,../data/generated/stylegan3-t-metfaces-1024x10...,0
2,../data/generated/stylegan3-t-metfaces-1024x10...,0
3,../data/generated/stylegan3-t-metfaces-1024x10...,0
4,../data/generated/stylegan3-t-metfaces-1024x10...,0
...,...,...
88146,../data/real/paul-albert-besnard_robert-de-mon...,1
88147,../data/real/nikolai-ge_christ-and-the-discipl...,1
88148,../data/real/paul-bril_view-of-a-port-1607.jpg,1
88149,../data/real/felix-vallotton_the-port-of-marse...,1


In [43]:
#train, validation = train_test_split(dataset.values, stratify=dataset.values[:, 1], test_size=.3, random_state = 1) 

# Split the dataset into two parts (train + validation, and test)
train_val_data, test = train_test_split(dataset.values, test_size=0.1, random_state=1)

# Split the train + validation part into training and validation sets
train, validation = train_test_split(train_val_data, test_size=0.1, random_state=1)

In [44]:
train_links = pd.DataFrame(train, columns = dataset.columns)
validation_links = pd.DataFrame(validation, columns = dataset.columns)
test_links = pd.DataFrame(test, columns = dataset.columns)

BUILDING DATA LOADERS

In [45]:
data_transforms = transforms.Compose([
                                transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               
     
])


batch_size = 32

train_set = ArtworkDataset( train_links, data_transforms)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, 
                               drop_last=False,num_workers=2)

validation_set = ArtworkDataset( validation_links, data_transforms)

validation_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=True, 
                               drop_last=False,num_workers=2)

test_set = ArtworkDataset( test_links,data_transforms)

test_loader = DataLoader(test_set,batch_size=batch_size, shuffle = True,
                              drop_last=False,num_workers=2)

TRAINING

In [46]:
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=5, min_delta=0.001):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.wait = 0
        self.best_loss = None
        self.early_stop = False
        # Define the directory for saving the model
        self.model_dir = os.path.join(os.getcwd(), 'resnet_model')  # Save in 'vit_model' folder in the current directory
        # Create the directory if it doesn't exist
        os.makedirs(self.model_dir, exist_ok=True)

    def __call__(self, current_loss):
        if self.best_loss == None:
            self.best_loss = current_loss
        elif (current_loss - self.best_loss) < -self.min_delta:
            self.best_loss = current_loss
            self.wait = 0
            
             # Save the model state
            model_path = os.path.join(self.model_dir, 'RealArt_vs_FakeArt_resnet50.pt')

            # Remove the old model if it exists in the same directory
            if os.path.exists(model_path):
                os.remove(model_path)
            
            # save the new model 
            torch.save(model.state_dict(), model_path)
        else:
            self.wait = self.wait + 1
            print(f"INFO: Early stopping counter {self.wait} of {self.patience}")
            if self.wait >= self.patience:
                self.early_stop = True

In [47]:
def fine_tune(model, train_loader, validation_loader, criterion, optimizer, scheduler, early_stop ,num_epochs = 100):
    best_model = copy.deepcopy(model)
    best_acc = 0.0
    best_epoch=0
    stop = False
    
    for epoch in range(1, num_epochs + 1):
        if stop:
            break
        print(f'Epoch {epoch}/{num_epochs}')
        print('-'*120)

        data_loader = None
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                data_loader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                data_loader = validation_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in tqdm(data_loader):
                
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs = nn.Softmax(dim = 1)(outputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / (len(data_loader) * data_loader.batch_size)
            epoch_acc = running_corrects.double() / (len(data_loader) * data_loader.batch_size)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_epoch = epoch
                best_model = copy.deepcopy(model)
               
                
            if phase == 'val':
                early_stop(epoch_loss)
                print('-'*120, end = '\n\n')
                stop=early_stop.early_stop
                
                
    print(f'Best val Acc: {best_acc:4f}')
    print(f'Best epoch: {best_epoch:03d}')

    # load the best model 
    return best_model         

In [48]:
if not 'RealArt_vs_FakeArt_resnet50.pt' in os.listdir('resnet_model'):
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.Adam(model.parameters(), lr=1e-3)
   scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
   early_stop = EarlyStopping(patience = 3, min_delta = 0.001)
   ImageFile.LOAD_TRUNCATED_IMAGES = True


   best_model_head = fine_tune(model, train_loader, validation_loader, criterion, optimizer, scheduler, early_stop, num_epochs = 30)
   
   # define the model path
   model_path = os.path.join('resnet_model', 'RealArt_vs_FakeArt_resnet50.pt')
   # Remove the old model if it exists
   if os.path.exists(model_path):
        os.remove(model_path)

   # Save the new best model head in the vit_model folder
   torch.save(best_model_head, model_path)

Epoch 1/30
------------------------------------------------------------------------------------------------------------------------


  0%|          | 0/2232 [00:00<?, ?it/s]

100%|██████████| 2232/2232 [01:07<00:00, 33.06it/s]


train Loss: 0.3910 Acc: 0.9345


100%|██████████| 248/248 [00:07<00:00, 34.39it/s]


...

val Loss: 0.3462 Acc: 0.9686
INFO: Early stopping counter 2 of 3
------------------------------------------------------------------------------------------------------------------------

Epoch 9/30
------------------------------------------------------------------------------------------------------------------------


100%|██████████| 2232/2232 [01:08<00:00, 32.36it/s]


train Loss: 0.3493 Acc: 0.9641


100%|██████████| 248/248 [00:07<00:00, 33.76it/s]

val Loss: 0.3478 Acc: 0.9665
INFO: Early stopping counter 3 of 3
------------------------------------------------------------------------------------------------------------------------

Best val Acc: 0.968624
Best epoch: 008


TESTING

In [49]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# funzione per il testing del modello
def test_model(model, test_loader):
    model.eval() # imposto il modello in modalità di valutazione
    test_loss = 0
    correct = 0
    pred_list = []
    true_list = []
    # inizializza la barra di avanzamento
    pbar = tqdm(total=len(test_loader))
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sommo il loss di ogni batch
            
            pred = output.argmax(dim=1, keepdim=True) # ottengo la predizione del modello
            pred_list.extend(pred.cpu().numpy()) # aggiungo la predizione alla lista
            true_list.extend(target.cpu().numpy()) # aggiungo il target alla lista
            
            correct += pred.eq(target.view_as(pred)).sum().item() # aggiorno il contatore di classificazioni corrette
            # aggiorna la barra di avanzamento
            pbar.update(1)
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    recall = recall_score(true_list, pred_list, average='macro') # calcolo la recall
    precision = precision_score(true_list, pred_list, average='macro') # calcolo la precision
    f1 = f1_score(true_list, pred_list, average='macro') # calcolo la F1 score
    auc = roc_auc_score(true_list, pred_list) # calcolo l'AUC
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%), Recall: {:.2f}%, Precision: {:.2f}%, F1: {:.2f}%, AUC: {:.2f}%\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy, recall*100, precision*100, f1*100, auc*100))
    
    return accuracy, recall, precision, f1, auc

In [50]:
accuracy,recall,precision,f1,auc = test_model(model,test_loader)

100%|██████████| 276/276 [00:07<00:00, 34.95it/s]


Test set: Average loss: 0.1071, Accuracy: 8534/8816 (96.80%), Recall: 96.59%, Precision: 96.94%, F1: 96.75%, AUC: 96.59%

